# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), July 25, 2018**

Let's get started by loading the `DataFrames` package.

In [1]:
using DataFrames

## Constructors and conversion

### Constructors

In this section, you'll see many ways to create a `DataFrame` using the `DataFrame()` constructor.

First, we could create an empty DataFrame,

In [2]:
DataFrame() # empty DataFrame

""


Or we could call the constructor using keyword arguments to add columns to the `DataFrame`.

In [3]:
DataFrame(A=1:3, B=rand(3), C=randstring.([3,3,3]))

,A,B,C
1,1,0.582939,eDh
2,2,0.899657,Uma
3,3,0.873748,RnO


We can create a `DataFrame` from a dictionary, in which case keys from the dictionary will be sorted to create the `DataFrame` columns.

In [4]:
x = Dict("A" => [1,2], "B" => [true, false], "C" => ['a', 'b'])
DataFrame(x)

,A,B,C
1,1,true,'a'
2,2,false,'b'


Rather than explicitly creating a dictionary first, as above, we could pass `DataFrame` arguments with the syntax of dictionary key-value pairs. 

Note that in this case, we use symbols to denote the column names and arguments are not sorted. For example, `:A`, the symbol, produces `A`, the name of the first column here:

In [5]:
DataFrame(:A => [1,2], :B => [true, false], :C => ['a', 'b'])

,A,B,C
1,1,true,'a'
2,2,false,'b'


Here we create a `DataFrame` from a vector of vectors, and each vector becomes a column.

In [6]:
DataFrame([rand(3) for i in 1:3])

,x1,x2,x3
1,0.291477,0.589915,0.321527
2,0.230679,0.0387716,0.539359
3,0.481537,0.494867,0.649062


 For now we can construct a single `DataFrame` from a `Vector` of atoms, creating a `DataFrame` with a single row. In future releases of DataFrames.jl, this will throw an error.

In [7]:
DataFrame(rand(3))

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] #DataFrame#62(::Bool, ::Type{T} where T, ::Array{Float64,1}, ::Array{Symbol,1}) at D:\Software\JULIA_PKG\v0.6\DataFrames\src\dataframe\dataframe.jl:154
 [3] DataFrames.DataFrame(::Array{Float64,1}) at D:\Software\JULIA_PKG\v0.6\DataFrames\src\dataframe\dataframe.jl:152
 [4] include_string(::String, ::String) at .\loading.jl:522
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at D:\Software\JULIA_PKG\v0.6\IJulia\src\execute_request.jl:158
 [6] (::Compat.#inner#14{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at D:\Software\JULIA_PKG\v0.6\Compat\src\Compat.jl:332
 [7] eventloop(::ZMQ.Socket) at D:\Software\JULIA_PKG\v0.6\IJulia\src\eventloop.jl:8
 [8] (::IJulia.##14#17)() at .\task.jl:335
while loading In[7], in expression starting on line 1


,x1,x2,x3
1,0.694614,0.305183,0.375302


Instead use a transposed vector if you have a vector of atoms (in this way you effectively pass a two dimensional array to the constructor which is supported).

In [8]:
DataFrame(transpose([1, 2, 3])) # permutedims in Julia 0.7

,x1,x2,x3
1,1,2,3


Pass a second argument to give the columns names.

In [9]:
DataFrame([1:3, 4:6, 7:9], [:A, :B, :C])

,A,B,C
1,1,4,7
2,2,5,8
3,3,6,9


Here we create a `DataFrame` from a matrix,

In [10]:
DataFrame(rand(3,4))

,x1,x2,x3,x4
1,0.957768,0.222008,0.569215,0.553819
2,0.0885923,0.49824,0.61231,0.985774
3,0.152064,0.804575,0.825947,0.653275


and here we do the same but also pass column names.

In [11]:
DataFrame(rand(3,4), Symbol.('a':'d'))

,a,b,c,d
1,0.188787,0.903936,0.500736,0.811463
2,0.341302,0.557099,0.41954,0.382749
3,0.0751378,0.430165,0.293501,0.253147


We can also construct an uninitialized DataFrame.

Here we pass column types, names and number of rows; we get `missing` in column :C because `Any >: Missing`.

In [12]:
DataFrame([Int, Float64, Any], [:A, :B, :C], 1)

,A,B,C
1,0,0.0,missing


Here we create a `DataFrame` where `:C` is #undef

In [13]:
DataFrame([Int, Float64, String], [:A, :B, :C], 1)

,A,B,C
1,-1,2.03553e-315,#undef


To initialize a `DataFrame` with column names, but no rows use

In [14]:
DataFrame([Int, Float64, String], [:A, :B, :C], 0) 

,A,B,C


This syntax gives us a quick way to create homogenous `DataFrame`.

In [15]:
DataFrame(Int, 3, 5)

,x1,x2,x3,x4,x5
1,166116144,165712048,166110096,169519056,165712048
2,164408368,167317968,111313296,111313680,164408368
3,111313296,141233040,141233200,141238528,141233232


This example is similar, but has nonhomogenous columns.

In [16]:
DataFrame([Int, Float64], 4)

,x1,x2
1,166692528,8.26715e-316
2,141234768,2.11841e-315
3,111411280,6.97793e-316
4,144117920,0.0


Finally, we can create a `DataFrame` by copying an existing `DataFrame`.

Note that `copy` creates a shallow copy.

In [17]:
y = DataFrame(x)
z = copy(x)
(x === y), (x === z), isequal(x, z)

(false, false, true)

### Conversion to a matrix

Let's start by creating a `DataFrame` with two rows and two columns.

In [18]:
x = DataFrame(x=1:2, y=["A", "B"])

,x,y
1,1,A
2,2,B


We can create a matrix by passing this `DataFrame` to `Matrix`.

In [19]:
Matrix(x)

2×2 Array{Any,2}:
 1  "A"
 2  "B"

This would work even if the `DataFrame` had some `missing`s:

In [20]:
x = DataFrame(x=1:2, y=[missing,"B"])

,x,y
1,1,missing
2,2,B


In [21]:
Matrix(x)

2×2 Array{Any,2}:
 1  missing
 2  "B"    

In the two previous matrix examples, Julia created matrices with elements of type `Any`. We can see more clearly that the type of matrix is inferred when we pass, for example, a `DataFrame` of integers to `Matrix`, creating a 2D `Array` of `Int64`s:

In [22]:
x = DataFrame(x=1:2, y=3:4)

,x,y
1,1,3
2,2,4


In [23]:
Matrix(x)

2×2 Array{Int64,2}:
 1  3
 2  4

In this next example, Julia correctly identifies that `Union` is needed to express the type of the resulting `Matrix` (which contains `missing`s).

In [24]:
x = DataFrame(x=1:2, y=[missing,4])

,x,y
1,1,missing
2,2,4


In [25]:
Matrix(x)

2×2 Array{Union{Int64, Missings.Missing},2}:
 1   missing
 2  4       

Note that we can't force a conversion of `missing` values to `Int`s!

In [26]:
Matrix{Int}(x)

LoadError: [91mcannot convert a DataFrame containing missing values to array (found for column y)[39m

### Handling of duplicate column names

We can pass the `makeunique` keyword argument to allow passing duplicate names (they get deduplicated)

In [27]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3; makeunique=true)

,a,a_2,a_1
1,1,2,3


Otherwise, duplicates will not be allowed in the future.

In [28]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] #make_unique#3(::Bool, ::Function, ::Array{Symbol,1}) at D:\Software\JULIA_PKG\v0.6\DataFrames\src\other\utils.jl:64
 [3] (::DataFrames.#kw##make_unique)(::Array{Any,1}, ::DataFrames.#make_unique, ::Array{Symbol,1}) at .\<missing>:0
 [4] #Index#6 at D:\Software\JULIA_PKG\v0.6\DataFrames\src\other\index.jl:12 [inlined]
 [5] (::Core.#kw#Type)(::Array{Any,1}, ::Type{DataFrames.Index}, ::Array{Symbol,1}) at .\<missing>:0
 [6] #DataFrame#52(::Bool, ::Type{T} where T, ::Pair{Symbol,Int64}, ::Vararg{Pair{Symbol,Int64},N} where N) at D:\Software\JULIA_PKG\v0.6\DataFrames\src\dataframe\dataframe.jl:126
 [7] DataFrames.DataFrame(::Pair{Symbol,Int64}, ::Pair{Symbol,Int64}, ::Pair{Symbol,Int64}) at D:\Software\JULIA_PKG\v0.6\DataFrames\src\dataframe\dataframe.jl:124
 [8] include_string(::String, ::String) at .\loading.jl:522
 [9] execute_request(::ZMQ.Socket, ::IJulia.Msg) at D:\Software\JULIA_PKG\v0.6\IJulia\src\execute_reque

,a,a_2,a_1
1,1,2,3


A constructor that is passed column names as keyword arguments is a corner case.
You cannot pass `makeunique` to allow duplicates here.

In [29]:
df = DataFrame(a=1, a=2, makeunique=true)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] #make_unique#3(::Bool, ::Function, ::Array{Symbol,1}) at D:\Software\JULIA_PKG\v0.6\DataFrames\src\other\utils.jl:64
 [3] (::DataFrames.#kw##make_unique)(::Array{Any,1}, ::DataFrames.#make_unique, ::Array{Symbol,1}) at .\<missing>:0
 [4] #Index#6 at D:\Software\JULIA_PKG\v0.6\DataFrames\src\other\index.jl:12 [inlined]
 [5] (::Core.#kw#Type)(::Array{Any,1}, ::Type{DataFrames.Index}, ::Array{Symbol,1}) at .\<missing>:0
 [6] #DataFrame#52(::Bool, ::Type{T} where T, ::Pair{Symbol,Int64}, ::Vararg{Pair{Symbol,#s8} where #s8,N} where N) at D:\Software\JULIA_PKG\v0.6\DataFrames\src\dataframe\dataframe.jl:126
 [7] DataFrames.DataFrame(::Pair{Symbol,Int64}, ::Pair{Symbol,Int64}, ::Pair{Symbol,Bool}) at D:\Software\JULIA_PKG\v0.6\DataFrames\src\dataframe\dataframe.jl:124
 [8] #DataFrame#61 at D:\Software\JULIA_PKG\v0.6\DataFrames\src\dataframe\dataframe.jl:145 [inlined]
 [9] (::Core.#kw#Type)(::Array{Any,1}, ::Type{DataFrame

,a,a_1,makeunique
1,1,2,true


Finallly observe that `nothing` is not printed when displaying a `DataFrame`:

In [30]:
DataFrame(x=[1, nothing], y=[nothing, "a"])

,x,y
1,1,
2,,a
